<a href="https://colab.research.google.com/github/robert-brosh23/SpotifyDuplicateFinder/blob/main/SpotifyDuplicateFinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
pip install spotipy --upgrade

In [75]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [76]:
scope = "playlist-modify-public"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id='82ef38fc0e954457b061319691ae6f09', client_secret='8369c21f75564e2aa239d6529bb5e79a', redirect_uri='http://localhost:3000', open_browser=False))

In [77]:
results = sp.current_user_playlists()
user = sp.current_user()['external_urls']['spotify']
print(user)
print(results)

https://open.spotify.com/user/froggthedogg
{'href': 'https://api.spotify.com/v1/users/froggthedogg/playlists?offset=0&limit=50', 'items': [{'collaborative': False, 'description': 'Playlist created by the tutorial on developer.spotify.com', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/2NCMaZuuG2Esj2xS8d3WKn'}, 'href': 'https://api.spotify.com/v1/playlists/2NCMaZuuG2Esj2xS8d3WKn', 'id': '2NCMaZuuG2Esj2xS8d3WKn', 'images': [{'height': 640, 'url': 'https://mosaic.scdn.co/640/ab67616d0000b2735b37091c34b90f98091a7422ab67616d0000b2736593537e6a1510219779f1a5ab67616d0000b2736eb7dbfb29e190d101b8a5f2ab67616d0000b273772da9eafd483d4958fa850f', 'width': 640}, {'height': 300, 'url': 'https://mosaic.scdn.co/300/ab67616d0000b2735b37091c34b90f98091a7422ab67616d0000b2736593537e6a1510219779f1a5ab67616d0000b2736eb7dbfb29e190d101b8a5f2ab67616d0000b273772da9eafd483d4958fa850f', 'width': 300}, {'height': 60, 'url': 'https://mosaic.scdn.co/60/ab67616d0000b2735b37091c34b90f98091a7422ab67616d0

In [78]:
def format_artists(artist_names):
    formatted = '('
    formatted+=', '.join(artist_names)
    formatted+=')'
    return formatted

In [79]:
def check_for_duplicates(item):
  playlist_id = item['id']
  next = "placeholder"
  track_list = []
  duplicate_list = []
  count = 0
  delete_items_map = []
  print("Checking {0} for duplicates...".format(item['name']))
  while(next!=None):
    results = sp.playlist_tracks(playlist_id=playlist_id,offset=count)
    items = results['items']
    for track_item in items:
      count+=1
      track = track_item['track']
      track_name = track['name']
      track_artists = track['artists']
      artist_names = []
      for track_artist in track_artists:
        artist_names.append(track_artist['name'])
      for passed_track in track_list:
        if track_name == passed_track[1] and artist_names == passed_track[2]:
          delete_items_map.append({"uri":track['uri'][14:],"positions":[count]})
          duplicate_list.append([passed_track[0],count,track_name,artist_names])
          break
      track_list.append([count,track_name,artist_names])
    next = results['next']
  if int(len(duplicate_list)) == 0:
    print("No duplicates found!")
  else:
    print(len(duplicate_list), "Duplicates found:")
    for duplicate_item in duplicate_list:
      formatted_artists = format_artists(duplicate_item[3])
      print("Number {0} and {1}: {2} {3}".format(duplicate_item[0],duplicate_item[1],duplicate_item[2],formatted_artists))

    ask_delete = input("Would you like to remove duplicates? Type 'Y' for yes.")
    if ask_delete == 'Y':
      offset = 1
      for delete_item_entry in delete_items_map:
        delete_item_entry['positions'][0]-=offset
        sp.playlist_remove_specific_occurrences_of_items(playlist_id=playlist_id,items=[delete_item_entry])
        offset+=1
      print("Removed {0} songs from playlist.".format(offset-1))


In [80]:
def check_for_duplicates_liked_songs():
  next = "placeholder"
  track_list = []
  duplicate_list = []
  count = 0
  delete_ids = []
  print("Checking Liked Songs for duplicates...")
  while(next!=None):
    results = sp.current_user_saved_tracks(offset=count)
    items = results['items']
    for track_item in items:
      count+=1
      track = track_item['track']
      track_name = track['name']
      track_artists = track['artists']
      artist_names = []
      for track_artist in track_artists:
        artist_names.append(track_artist['name'])
      for passed_track in track_list:
        if track_name == passed_track[1] and artist_names == passed_track[2]:
          delete_ids.append(track['id'])
          duplicate_list.append([passed_track[0],count,track_name,artist_names])
          break
      track_list.append([count,track_name,artist_names])
    next = results['next']
  if int(len(duplicate_list)) == 0:
    print("No duplicates found!")
  else:
    print(len(duplicate_list), "Duplicates found:")
    for duplicate_item in duplicate_list:
      formatted_artists = format_artists(duplicate_item[3])
      print("Number {0} and {1}: {2} {3}".format(duplicate_item[0],duplicate_item[1],duplicate_item[2],formatted_artists))

    ask_delete = input("Would you like to remove duplicates? Type 'Y' for yes.")
    if ask_delete == 'Y':
      offset = 1
      sp.current_user_saved_tracks_delete(tracks=delete_ids)
      print("Removed {0} songs from playlist.".format(len(delete_ids)))

In [81]:
count = 2
items = results['items']
print("Found playlists:")
index_item_map = {1:'Liked Songs'}
print("1. Liked Songs".format(count, name))
for item in items:
  if item['owner']['external_urls']['spotify'] == user:
    name = item['name']
    print("{0}. {1}".format(count, name))
    index_item_map[count]=item
    count=count+1

Found playlists:
1. Liked Songs
2. spotify api test playlist
3. Nostalgia
4. Covers
5. Roadtrip!!!
6. Rise up 2
7. For Willow
8. HYPErpop
9. Strong
10. mocktail
11. Coding (Instrumental)
12. Folk Punk/Indie Rock/Midwest Emo
13. that's tough, bro
14. C O R O N A V I R U S
15. Coffee, or Orange Juice?
16. H Y P E M A N
17. Weeb 2
18. Semi-hipster family friendly mix
19. Weeb
20. rise up


In [82]:
saved_tracks = sp.current_user_saved_tracks()['items']
print(saved_tracks)

[{'added_at': '2023-09-06T05:36:07Z', 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0gqljtE0ZzPxEpjmHeNvTx'}, 'href': 'https://api.spotify.com/v1/artists/0gqljtE0ZzPxEpjmHeNvTx', 'id': '0gqljtE0ZzPxEpjmHeNvTx', 'name': 'Little Big League', 'type': 'artist', 'uri': 'spotify:artist:0gqljtE0ZzPxEpjmHeNvTx'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', 'MD', 'UA', 'AL', 'BA', 'HR', 'ME', 'MK', 'RS', 'SI', 'KR', 'BD', 'PK', 'LK', 'GH', 'KE', 'NG', 'TZ', 'UG', 'AG'

In [83]:
playlist_number = input("What number playlist would you like to check? Type 'ALL' to check all playlists.")
if playlist_number == 'ALL':
  print("Checking ALL playlists...")
  for key in index_item_map:
    if key == 1:
      check_for_duplicates_liked_songs()
    else:
      item = index_item_map[key]
      check_for_duplicates(item)
  print("Finished checking all playlists.")
elif playlist_number == '1':
  print("Checking 1. Liked Songs")
  check_for_duplicates_liked_songs()
else:
  item = index_item_map.get(int(playlist_number))
  name = item['name']
  print("Checking {0}. {1}".format(playlist_number, name))
  check_for_duplicates(item)


What number playlist would you like to check? Type 'ALL' to check all playlists.5
Checking 5. Roadtrip!!!
Checking Roadtrip!!! for duplicates...
No duplicates found!
